## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches
import joblib
import os

In [3]:
df = pd.read_csv('../data/raw/final_perfume_data.csv', encoding='ISO-8859-1')
df.head()

,Name,Brand,Description,Notes,Image URL
0,Tihota Eau de Parfum,Indult,"Rapa Nui for sugar, Tihota is, quite simply, ...","Vanilla bean, musks",https://static.luckyscent.com/images/products/...
1,Sola Parfum,Di Ser,A tribute to the expanse of space extending f...,"Lavender, Yuzu, Lemongrass, Magnolia, Geraniu...",https://static.luckyscent.com/images/products/...
2,Kagiroi Parfum,Di Ser,An aromatic ode to the ancient beauty of Japa...,"Green yuzu, green shikuwasa, sansho seed, cor...",https://static.luckyscent.com/images/products/...
3,Velvet Fantasy Eau de Parfum,Montale,Velvet Fantasy is a solar fragrance where cit...,"tangerine, pink pepper, black coffee, leat...",https://static.luckyscent.com/images/products/...
4,A Blvd. Called Sunset Eau de Parfum,A Lab on Fire,There's no way A Lab On Fire could relocate t...,"Bergamot, almond, violet, jasmine, leather, s...",https://static.luckyscent.com/images/products/...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         2191 non-null   object
 1   Brand        2191 non-null   object
 2   Description  2191 non-null   object
 3   Notes        2111 non-null   object
 4   Image URL    2191 non-null   object
dtypes: object(5)
memory usage: 85.7+ KB


## Data Preprocessing

In [5]:
# Creating a new df for cleaning
cleaned_df = df.copy()

In [6]:
# Removing null values from the dataset
print(cleaned_df.isnull().sum())
cleaned_df.dropna(inplace=True)

Name            0
Brand           0
Description     0
Notes          80
Image URL       0
dtype: int64


In [7]:
# Removing the column for description and imageurl
cleaned_df.drop(columns=['Description', 'Image URL'], inplace=True)

In [11]:
# Lowering the column names and the values of the column
cleaned_df.columns = cleaned_df.columns.str.lower()
cleaned_df['name'] = cleaned_df['name'].str.lower()
cleaned_df['brand'] = cleaned_df['brand'].str.lower()
cleaned_df['notes'] = cleaned_df['notes'].str.lower()


In [14]:
# Removing the duplicate rows from the dataset
print(cleaned_df.duplicated().sum())

0


In [ ]:
# Spliting the note values
